# with updated lodes data

In [1]:
import pandas as pd
import geopandas as gpd
import json
import requests
import numpy as np
from shapely.geometry import Point

In [19]:
def traveltimeTransit(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
    r = requests.get(request)
    data=r.json()
    # try:
    #     legs = pd.DataFrame(data['plan']['itineraries'][0]['legs'])
    # except:
    #     return np.nan
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return (data['plan']['itineraries'])
    else:
        return np.nan

In [3]:
def traveltimeTransit2(df, i, latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
    r = requests.get(request)
    data=r.json()
 
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        df.loc[i, 'transit_time'] = int(data['plan']['itineraries'][0]['duration'])
        df.loc[i, 'transfers'] = int(data['plan']['itineraries'][0]['transfers'])
        legs = pd.DataFrame(data['plan']['itineraries'][0]['legs'])
        # df.loc[i, 'trips'] = np.array(legs['tripId'].dropna().to_list())
        # df.loc[i, 'trips'] = df.apply(lambda r: tuple(legs['tripId'].dropna()), axis=1).apply(np.array)
        df.loc[i, 'trip1'] = legs['tripId'].dropna().iloc[0].split(':')[1]
        try:
            df.loc[i, 'trip2'] = legs['tripId'].dropna().iloc[0].split(':')[1]
        except:pass
        try:
            df.loc[i, 'trip3'] = legs['tripId'].dropna().iloc[0].split(':')[2]
        except:pass
        df.loc[i, 'route1'] = legs.route[legs.route != ''].iloc[0]
        try:
            df.loc[i, 'route2'] = legs.route[legs.route != ''].iloc[1]
        except:pass
        try:
            df.loc[i, 'route3'] = legs.route[legs.route != ''].iloc[2]
        except:pass
        df.loc[i, 'bus_capacity_upto_2hours'] = len(data['plan']['itineraries'])*30

        
    else:
        df.loc[i, 'transit_time'] =  np.nan
        df.loc[i, 'transfers'] = np.nan
        # df.loc[i, 'trips'] = np.nan
        df.loc[i, 'trip1'] =  np.nan
        df.loc[i, 'trip2'] =  np.nan
        df.loc[i, 'trip3'] =  np.nan
        df.loc[i, 'route1'] =  np.nan
        df.loc[i, 'route2'] =  np.nan
        df.loc[i, 'route3'] =  np.nan
        df.loc[i, 'bus_capacity_upto_2hours'] = np.nan

    
    

In [4]:
def traveltimeDrive(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK,CAR&date=05-04-2021&time="\
    +time+"&maxWalkDistance=750"
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [5]:
def traveltimeWalk(latHome, lonHome,latWork,lonWork, time)->int:
    request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(latHome)\
    + ","+str(lonHome)+"&toPlace="+str(latWork) + ","+str(lonWork)+ "&"+"&mode=WALK&date=05-04-2021&time="\
    +time+"&maxWalkDistance=50000"
    r = requests.get(request)
    data=r.json()
    # print(data['plan'])
    if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
        return int(data['plan']['itineraries'][0]['duration'])
    else:
        return np.nan

In [6]:
def get_stop_url(geom):
    # print(df)
    # geom = df.geometry
    initial='http://127.0.0.1:5000/nearest/v1/driving/'
    end='?number=1'
    point = str(geom.x)+","+str(geom.y)
    requests2 = []
    requests2.append(initial+point+end)
    return requests2
    

In [7]:
import time
def nearby_points(reqs):
    url=reqs[0]
    #to remove any connection timeouts and port usage exceptions
    # r=None
    # l=None
    try:
        r=requests.get(url)
        data=r.json()
        l=Point(data['waypoints'][0]['location'])
        return l
    except:
        time.sleep(60)
        
    # return l

## input: lodes data(block-wise), tracts

In [8]:
lodes_ham = pd.read_csv('hamilton_lodes_2019.csv', dtype={"TRACTCE20_home":"string", "TRACTCE20_work":"string"})
# lodes_ham.TRACTCE20_home = lodes_ham.TRACTCE20_home.astype(str)

tracts  = gpd.read_file('../Tennessee Census Tracts/tl_2016_47_tract.shp')
tracts = tracts[tracts['COUNTYFP'] == '065']
tracts.TRACTCE = tracts.TRACTCE.astype(str)

In [9]:
tracts.TRACTCE = tracts.TRACTCE.astype(str)

In [46]:
tracts

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
48,47,065,010411,47065010411,104.11,Census Tract 104.11,G5020,S,17103362,37850,+35.1876924,-085.2117517,"POLYGON ((-85.24658 35.18283, -85.24651 35.183..."
49,47,065,001800,47065001800,18,Census Tract 18,G5020,S,5595289,0,+34.9988768,-085.3342208,"POLYGON ((-85.34730 34.98896, -85.34728 34.989..."
50,47,065,011411,47065011411,114.11,Census Tract 114.11,G5020,S,12264968,3031800,+35.0958357,-085.2149270,"POLYGON ((-85.27004 35.09113, -85.26977 35.091..."
51,47,065,001900,47065001900,19,Census Tract 19,G5020,S,8494824,30874,+35.0037462,-085.3148911,"POLYGON ((-85.33003 35.01638, -85.33002 35.016..."
52,47,065,010700,47065010700,107,Census Tract 107,G5020,S,4693150,0,+35.0956240,-085.3014084,"POLYGON ((-85.32080 35.07814, -85.32075 35.078..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,47,065,012400,47065012400,124,Census Tract 124,G5020,S,2792792,267513,+35.0440253,-085.2978673,"POLYGON ((-85.30756 35.03952, -85.30741 35.039..."
1445,47,065,010800,47065010800,108,Census Tract 108,G5020,S,4772118,0,+35.1026431,-085.3079013,"POLYGON ((-85.32444 35.08866, -85.32441 35.089..."
1446,47,065,011447,47065011447,114.47,Census Tract 114.47,G5020,S,23415253,4586370,+35.1254731,-085.0982693,"POLYGON ((-85.13968 35.15264, -85.13925 35.153..."
1447,47,065,980200,47065980200,9802,Census Tract 9802,G5020,S,26731896,0,+35.0892045,-085.1312515,"POLYGON ((-85.16965 35.09932, -85.16963 35.099..."


In [10]:
hw_tracts_geom = lodes_ham.groupby(['TRACTCE20_home', 'TRACTCE20_work']).sum().sort_values('total_jobs', ascending=False).reset_index()[['TRACTCE20_home', 'TRACTCE20_work', 'total_jobs']].dropna()

In [11]:
hw_tracts_geom = hw_tracts_geom.merge(tracts[['TRACTCE', 'geometry']], left_on='TRACTCE20_home', right_on='TRACTCE', how='left').merge(tracts[['TRACTCE', 'geometry']], left_on='TRACTCE20_work', right_on='TRACTCE', how='left', suffixes=('_home', '_work'))

In [12]:
hw_tracts_geom['home_geom_centroid'] = hw_tracts_geom.geometry_home.apply(lambda x : x.centroid)
hw_tracts_geom['work_geom_centroid'] = hw_tracts_geom.geometry_work.apply(lambda x : x.centroid)

In [13]:
hw_tracts_geom['hc_url'] = hw_tracts_geom.home_geom_centroid.apply(get_stop_url)
hw_tracts_geom['wc_url'] = hw_tracts_geom.work_geom_centroid.apply(get_stop_url)

In [14]:
hw_tracts_geom['hc_nearby']=hw_tracts_geom.hc_url.apply(nearby_points)
hw_tracts_geom['wc_nearby']=hw_tracts_geom.wc_url.apply(nearby_points)

In [15]:
hw_tracts_geom['hc_nearby_x'] = hw_tracts_geom.hc_nearby.apply(lambda pt : pt.x)
hw_tracts_geom['hc_nearby_y'] = hw_tracts_geom.hc_nearby.apply(lambda pt : pt.y)

hw_tracts_geom['wc_nearby_x'] = hw_tracts_geom.wc_nearby.apply(lambda pt : pt.x)
hw_tracts_geom['wc_nearby_y'] = hw_tracts_geom.wc_nearby.apply(lambda pt : pt.y)

# hw_tracts_geom['hc_nearby_x'] = hw_tracts_geom.home_geom_centroid.apply(lambda pt : pt.x)
# hw_tracts_geom['hc_nearby_y'] = hw_tracts_geom.home_geom_centroid.apply(lambda pt : pt.y)

# hw_tracts_geom['wc_nearby_x'] = hw_tracts_geom.work_geom_centroid.apply(lambda pt : pt.x)
# hw_tracts_geom['wc_nearby_y'] = hw_tracts_geom.work_geom_centroid.apply(lambda pt : pt.y)

In [16]:
for i in range(hw_tracts_geom.shape[0]):
    traveltimeTransit2(hw_tracts_geom, i, hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'drive_time_fn'] = traveltimeDrive(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'walk_time_fn'] = traveltimeWalk(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'transit_score'] = np.divide(hw_tracts_geom.loc[i, 'drive_time_fn'], hw_tracts_geom.loc[i, 'transit_time'])

['1:205758020' '1:206772020' '1:204957020' '1:205167020']
['1:205758020' '1:206772020' '1:204957020' '1:205167020']
['1:204957020']
['1:205758020' '1:206772020' '1:204957020' '1:205167020']
['1:206791020' '1:204921020']
['1:205186020']
['1:205758020' '1:206772020' '1:204957020' '1:205167020']
['1:205758020' '1:206772020' '1:204957020' '1:205167020']
['1:205758020']
['1:206341020' '1:205075020']
['1:206297020' '1:205186020']
['1:204957020' '1:205167020']
['1:205758020' '1:205517020']
['1:205186020']
['1:205758020']
['1:204942020' '1:205057020']
['1:206341020' '1:206296020' '1:204921020']
['1:206791020' '1:204921020']
['1:206341020' '1:205075020']
['1:205186020']
['1:205388020']
['1:206297020' '1:205186020' '1:204909020']
['1:206791020' '1:204921020']
['1:205758020' '1:205517020']
['1:205758020']
['1:205758020']
['1:206341020' '1:205167020']
['1:204942020' '1:205057020']
['1:205758020' '1:206772020' '1:204957020' '1:205167020']
['1:206062020' '1:206084020']
['1:205186020' '1:204909020']


In [ ]:
for i in range(hw_tracts_geom.shape[0]):
    transit = traveltimeTransit(hw_tracts_geom, i, hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'transit_time_fn'] = transit[0]
    hw_tracts_geom.loc[i, 'legs'] = transit[1]
    drive = traveltimeDrive(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    walk = traveltimeWalk(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_tracts_geom.loc[i, 'transit_score'] = np.divide(hw_tracts_geom.loc[i, 'drive_time_fn'], hw_tracts_geom.loc[i, 'transit_time_fn'])

In [24]:
#saving to csv
hw_tracts_geom = hw_tracts_geom.rename({'ratio':'transit_score'}, axis=1)
hw_tracts_geom[['TRACTCE_home', 'TRACTCE_work', 'geometry_home', 'geometry_work', 'total_jobs', 'transit_time_fn', 'drive_time_fn', 'walk_time_fn', 'transit_score']].to_csv('transit_score_tracts_with_geom.csv', index=False)
# .to_csv('hamilton_tracts_transit_score.csv')

In [58]:
hw_tracts_geom.groupby('TRACTCE_home').mean().reset_index()[['TRACTCE_home', 'transit_score']].to_csv('tract_transit_score_home.csv', index=False)
hw_tracts_geom.groupby('TRACTCE_work').mean().reset_index()[['TRACTCE_work', 'transit_score']].to_csv('tract_transit_score_work.csv', index=False)

In [52]:
hw_tracts_geom.dropna(subset=['transit_score']).groupby('TRACTCE20_home').mean().reset_index().merge(tracts[['TRACTCE', 'geometry']], left_on='TRACTCE20_home', right_on='TRACTCE', how='left').to_csv('tracts_lodes_ts.csv', index=False)
# .to_csv('tracts_ts_filtered_new.csv', index=False)

In [57]:
hw_tracts_geom.to_csv('lodes_all_tracts.csv', index=False)
# [hw_tracts_geom['TRACTCE20_home'] == hw_tracts_geom['TRACTCE20_work']].transit_score.dropna()

### testing

In [41]:
i=46
temp_df = hw_tracts_geom
traveltimeTransit(temp_df.iloc[i].hc_nearby_y, temp_df.iloc[i].hc_nearby_x, temp_df.iloc[i].wc_nearby_y, temp_df.iloc[i].wc_nearby_x, '08:00am')
# legs = pd.DataFrame(k[2]['legs'])
# print(legs['tripId'].dropna().iloc[0].split(':')[1])
# print(k)

[{'duration': 2956,
  'startTime': 1651667297000,
  'endTime': 1651670253000,
  'walkTime': 716,
  'transitTime': 1716,
  'waitingTime': 524,
  'walkDistance': 907.0543533772843,
  'walkLimitExceeded': False,
  'elevationLost': 0.0,
  'elevationGained': 0.0,
  'transfers': 1,
  'legs': [{'startTime': 1651667297000,
    'endTime': 1651667530000,
    'departureDelay': 0,
    'arrivalDelay': 0,
    'realTime': False,
    'distance': 294.652,
    'pathway': False,
    'mode': 'WALK',
    'route': '',
    'agencyTimeZoneOffset': -14400000,
    'interlineWithPreviousLeg': False,
    'from': {'name': 'Origin',
     'lon': -85.282381,
     'lat': 34.994132,
     'departure': 1651667297000,
     'orig': '',
     'vertexType': 'NORMAL'},
    'to': {'name': 'DODDS + 41ST',
     'stopId': '1:1501',
     'stopCode': '3101',
     'lon': -85.284275,
     'lat': 34.995335,
     'arrival': 1651667530000,
     'departure': 1651667531000,
     'stopIndex': 11,
     'stopSequence': 12,
     'vertexType': 

In [34]:
i=46
# temp_df = hw_tracts_geom
# traveltimeTransit2(temp_df, i, temp_df.iloc[i].hc_nearby_y, temp_df.iloc[i].hc_nearby_x, temp_df.iloc[i].wc_nearby_y, temp_df.iloc[i].wc_nearby_x, '10:00am')

# print(temp_df.iloc[i])
# '''
time='08:00am'
# traveltimeTransit(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(hw_tracts_geom.iloc[i].hc_nearby_y)\
    + ","+str(hw_tracts_geom.iloc[i].hc_nearby_x)+"&toPlace="+str(hw_tracts_geom.iloc[i].wc_nearby_y) + ","+str(hw_tracts_geom.iloc[i].wc_nearby_x)+ "&"+"&mode=WALK,TRANSIT&date=05-04-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
r = requests.get(request)
data=r.json()
# if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
    # print (data['plan']['itineraries'][0])

print('itineraries', data['plan']['itineraries'][1]['legs'][1]['mode'])
legs = pd.DataFrame(data['plan']['itineraries'][0]['legs'])
print(legs.route[legs.route != ''].iloc[0])
print("transfers: ", data['plan']['itineraries'][0]['transfers'])
trips = []
trips = (list(legs['tripId'].dropna()))
print(legs['route'].dropna().iloc[0])
# print(len(data['plan']['itineraries']))
# for it in data['plan']['itineraries'][2:]:
#     legs = it['legs']
#     for leg in legs:
#         print(leg['mode'])
# for leg in legs:
#     print (leg)
#     if leg['tripId']:
#         trips.append(leg['tripId'])
# print(trips)
# for leg in legs:
#     print("distance: ", leg['distance'])
#     print("mode: ", leg['mode'])
#     if leg['route']:
#         print("route: ", leg['route'])
#     try:
#         print("tripId: ", leg['tripId'])
#     except:
#         pass
#     try:
#         print("from stop: ", leg['from']['stopId'])
#     except:pass
#     try:
#         print("from stop: ", leg['to']['stopId'])
#     except:pass
#     print('\n')
    
# '''

itineraries BUS
9
transfers:  1



## for blocks

In [23]:
blocks  = gpd.read_file('../Tennessee Census Block/tl_2020_47_tabblock20.shp')

In [28]:
lodes_ham.groupby(['h_geocode', 'w_geocode']).sum().reset_index().sort_values('total_jobs', ascending=False)
lodes_ham.GEOID20_home = lodes_ham.GEOID20_home.astype(str)
lodes_ham.GEOID20_work = lodes_ham.GEOID20_work.astype(str)

In [34]:
import geopandas as gpd

hw_blocks = gpd.read_file('hamilton_lodes_2019.csv', geometry=['geometry_home', 'geometry_work'])
# .groupby(['h_geocode', 'w_geocode']).sum().sort_values('total_jobs', ascending=False).reset_index()

In [55]:
# type(hw_blocks.geometry_home.iloc[0])
from shapely import wkt
 
hw_blocks['geometry_home'] = hw_blocks['geometry_home'].apply(wkt.loads)
hw_blocks['geometry_work'] = hw_blocks['geometry_work'].apply(wkt.loads)

hw_blocks = hw_blocks.drop('geometry', axis=1)
#  = hw_blocks.drop('geometry', axis=1)


In [29]:
hw_blocks = hw_blocks.merge(lodes_ham[['GEOID20_home', 'geometry_home']], left_on='h_geocode', right_on='GEOID20_home', how='left')

In [35]:
hw_blocks_geom_drp =  hw_blocks.reset_index(drop=True)

In [36]:
hw_blocks_geom_drp

,w_geocode,h_geocode,total_jobs,SA01,SA02,SA03,SE01,SE02,SE03,SI01,...,TRACTCE20_home,BLOCKCE20_home,GEOID20_home,geometry_home,COUNTYFP20_work,TRACTCE20_work,BLOCKCE20_work,GEOID20_work,geometry_work,geometry
0,470650004001044,470650113214000,55,13,28,14,7,21,27,0,...,011321,4000,470650113214000,"POLYGON ((-85.144618 35.027176, -85.142107 35....",065,000400,1044,470650004001044,"POLYGON ((-85.269936 35.042088, -85.2698719999...",None
1,470650004001044,470650113214000,52,13,26,13,5,20,27,0,...,011321,4000,470650113214000,"POLYGON ((-85.144618 35.027176, -85.142107 35....",065,000400,1044,470650004001044,"POLYGON ((-85.269936 35.042088, -85.2698719999...",None
2,470650004001044,470650114474000,40,4,25,11,1,17,22,0,...,011447,4000,470650114474000,"POLYGON ((-85.109624 35.147346, -85.109237 35....",065,000400,1044,470650004001044,"POLYGON ((-85.269936 35.042088, -85.2698719999...",None
3,470650004001044,470650113212001,29,7,16,6,0,15,14,0,...,011321,2001,470650113212001,"POLYGON ((-85.144706 35.014255999999996, -85.1...",065,000400,1044,470650004001044,"POLYGON ((-85.269936 35.042088, -85.2698719999...",None
4,470650004001044,470650113213000,25,6,13,6,0,8,17,0,...,011321,3000,470650113213000,"POLYGON ((-85.134131 35.018297, -85.132252 35....",065,000400,1044,470650004001044,"POLYGON ((-85.269936 35.042088, -85.2698719999...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37522,470650023001056,470650113142005,1,0,0,1,0,1,0,1,...,011314,2005,470650113142005,"POLYGON ((-85.09944999999999 35.027822, -85.09...",065,002300,1056,470650023001056,POLYGON ((-85.29514999999999 34.99715899999999...,None
37523,470650023001056,470650104131003,1,0,0,1,0,0,1,1,...,010413,1003,470650104131003,"POLYGON ((-85.177939 35.161048, -85.177904 35....",065,002300,1056,470650023001056,POLYGON ((-85.29514999999999 34.99715899999999...,None
37524,470650023001056,470650104333000,1,0,1,0,0,0,1,1,...,010433,3000,470650104333000,"POLYGON ((-85.25809 35.121561, -85.25806 35.12...",065,002300,1056,470650023001056,POLYGON ((-85.29514999999999 34.99715899999999...,None
37525,470650023001056,470650118001011,1,0,1,0,0,0,1,1,...,011800,1011,470650118001011,"POLYGON ((-85.26884299999999 34.991911, -85.26...",065,002300,1056,470650023001056,POLYGON ((-85.29514999999999 34.99715899999999...,None


In [53]:
hw_blocks_geom_drp['home_geom_centroid'] = hw_blocks_geom_drp.geometry_home.apply(lambda x : x.centroid)
hw_blocks_geom_drp['work_geom_centroid'] = hw_blocks_geom_drp.geometry_work.apply(lambda x : x.centroid)

AttributeError: 'str' object has no attribute 'centroid'

In [33]:
hw_blocks_geom_drp['hc_url'] = hw_blocks_geom_drp.home_geom_centroid.apply(get_stop_url)
hw_blocks_geom_drp['wc_url'] = hw_blocks_geom_drp.work_geom_centroid.apply(get_stop_url)

AttributeError: 'GeoDataFrame' object has no attribute 'home_geom_centroid'

In [ ]:
hw_blocks_geom_drp['hc_nearby']=hw_blocks_geom_drp.hc_url.apply(nearby_points)
hw_blocks_geom_drp['wc_nearby']=hw_blocks_geom_drp.wc_url.apply(nearby_points)

In [ ]:
hw_blocks_geom_drp = hw_blocks_geom_drp.dropna(subset=['hc_nearby', 'wc_nearby'])

In [ ]:
hw_blocks_geom_drp['hc_nearby_x'] = hw_blocks_geom_drp.hc_nearby.apply(lambda pt : pt.x)
hw_blocks_geom_drp['hc_nearby_y'] = hw_blocks_geom_drp.hc_nearby.apply(lambda pt : pt.y)

hw_blocks_geom_drp['wc_nearby_x'] = hw_blocks_geom_drp.wc_nearby.apply(lambda pt : pt.x)
hw_blocks_geom_drp['wc_nearby_y'] = hw_blocks_geom_drp.wc_nearby.apply(lambda pt : pt.y)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [ ]:
for i in range(hw_blocks_geom_drp.shape[0]):
    hw_blocks_geom_drp.loc[i, 'transit_time_fn'] = traveltimeTransit(hw_blocks_geom_drp.iloc[i].hc_nearby_y, hw_blocks_geom_drp.iloc[i].hc_nearby_x, hw_blocks_geom_drp.iloc[i].wc_nearby_y, hw_blocks_geom_drp.iloc[i].wc_nearby_x, '10:00am')
    hw_blocks_geom_drp.loc[i, 'drive_time_fn'] = traveltimeDrive(hw_blocks_geom_drp.iloc[i].hc_nearby_y, hw_blocks_geom_drp.iloc[i].hc_nearby_x, hw_blocks_geom_drp.iloc[i].wc_nearby_y, hw_blocks_geom_drp.iloc[i].wc_nearby_x, '10:00am')
    # hw_tracts_geom.loc[i, 'walk_time_fn'] = traveltimeWalk(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
    hw_blocks_geom_drp.loc[i, 'transit_score'] = np.divide(hw_blocks_geom_drp.loc[i, 'drive_time_fn'], hw_blocks_geom_drp.loc[i, 'transit_time_fn'])

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is

In [ ]:
hw_blocks_geom_drp.dropna(subset=['h_geocode']).reset_index()

,index,w_geocode,h_geocode,total_jobs,SA01,SA02,SA03,SE01,SE02,SE03,...,wc_url,hc_nearby,wc_nearby,hc_nearby_x,hc_nearby_y,wc_nearby_x,wc_nearby_y,transit_time_fn,drive_time_fn,transit_score
0,0,470650004001044,470650113214000,55,13,28,14,7,21,27,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.139669 35.030262),POINT (-85.269313 35.04185),-85.139669,35.030262,-85.269313,35.041850,NaN,1689.0,NaN
1,1,470650004001044,470650113214000,52,13,26,13,5,20,27,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.139669 35.030262),POINT (-85.269313 35.04185),-85.139669,35.030262,-85.269313,35.041850,NaN,1689.0,NaN
2,2,470650004001044,470650114474000,40,4,25,11,1,17,22,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.085202 35.148474),POINT (-85.269313 35.04185),-85.085202,35.148474,-85.269313,35.041850,NaN,3010.0,NaN
3,3,470650004001044,470650113212001,29,7,16,6,0,15,14,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.136971 35.015931),POINT (-85.269313 35.04185),-85.136971,35.015931,-85.269313,35.041850,NaN,2029.0,NaN
4,4,470650004001044,470650113213000,25,6,13,6,0,8,17,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.12762 35.023764),POINT (-85.269313 35.04185),-85.127620,35.023764,-85.269313,35.041850,NaN,1982.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37514,37522,470650023001056,470650113142005,1,0,0,1,0,1,0,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.093081 35.026971),POINT (-85.294063 34.996814),-85.093081,35.026971,-85.294063,34.996814,NaN,NaN,NaN
37515,37523,470650023001056,470650104131003,1,0,0,1,0,0,1,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.176018 35.15834),POINT (-85.294063 34.996814),-85.176018,35.158340,-85.294063,34.996814,NaN,NaN,NaN
37516,37524,470650023001056,470650104333000,1,0,1,0,0,0,1,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.252867 35.124041),POINT (-85.294063 34.996814),-85.252867,35.124041,-85.294063,34.996814,NaN,NaN,NaN
37517,37525,470650023001056,470650118001011,1,0,1,0,0,0,1,...,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.266872 34.991584),POINT (-85.294063 34.996814),-85.266872,34.991584,-85.294063,34.996814,NaN,NaN,NaN


In [ ]:
i=15
time='10:00am'
# traveltimeTransit(hw_tracts_geom.iloc[i].hc_nearby_y, hw_tracts_geom.iloc[i].hc_nearby_x, hw_tracts_geom.iloc[i].wc_nearby_y, hw_tracts_geom.iloc[i].wc_nearby_x, '10:00am')
request= "http://localhost:8080/otp/routers/default/plan?fromPlace="+str(hw_blocks_geom_drp.iloc[i].hc_nearby_y)\
    + ","+str(hw_blocks_geom_drp.iloc[i].hc_nearby_x)+"&toPlace="+str(hw_blocks_geom_drp.iloc[i].wc_nearby_y) + ","+str(hw_blocks_geom_drp.iloc[i].wc_nearby_x)+ "&"+"&mode=WALK,TRANSIT&date=05-01-2022&time="\
    +time+"&maxWalkDistance=750"
    # print(request)
r = requests.get(request)
data=r.json()
if 'plan' in data.keys() and 'itineraries' in data['plan'].keys():
    # print (data['plan']['itineraries'][0])
    legs = pd.DataFrame(data['plan']['itineraries'][0]['legs'])
    print(legs.route)
    print("transfers: ", data['plan']['itineraries'][0]['transfers'])
# for leg in legs:
#     print("distance: ", leg['distance'])
#     print("mode: ", leg['mode'])
#     if leg['route']:
#         print("route: ", leg['route'])
#     try:
#         print("tripId: ", leg['tripId'])
#     except:
#         pass
#     try:
#         print("from stop: ", leg['from']['stopId'])
#     except:pass
#     try:
#         print("from stop: ", leg['to']['stopId'])
#     except:pass
#     print('\n')
data
    

0       
1    10G
2       
Name: route, dtype: object
transfers:  0


{'requestParameters': {'mode': 'WALK,TRANSIT',
  'date': '05-01-2022',
  'fromPlace': '35.052118,-85.25549',
  'toPlace': '35.04185,-85.269313',
  'time': '10:00am',
  'maxWalkDistance': '750'},
 'plan': {'date': 1651413600000,
  'from': {'name': 'Origin',
   'lon': -85.25549,
   'lat': 35.052118,
   'orig': '',
   'vertexType': 'NORMAL'},
  'to': {'name': 'Destination',
   'lon': -85.269313,
   'lat': 35.04185,
   'orig': '',
   'vertexType': 'NORMAL'},
  'itineraries': [{'duration': 1095,
    'startTime': 1651416563000,
    'endTime': 1651417658000,
    'walkTime': 294,
    'transitTime': 799,
    'waitingTime': 2,
    'walkDistance': 371.21200751547326,
    'walkLimitExceeded': False,
    'elevationLost': 0.0,
    'elevationGained': 0.0,
    'transfers': 0,
    'legs': [{'startTime': 1651416563000,
      'endTime': 1651416653000,
      'departureDelay': 0,
      'arrivalDelay': 0,
      'realTime': False,
      'distance': 113.32400000000001,
      'pathway': False,
      'mode': 'W

In [ ]:
#tract mean score from block score
hw_blocks_geom_drp2 = hw_blocks_geom_drp
# .groupby('TRACTCE20').mean()

In [ ]:
#storing to csv
hw_blocks_geom_drp.groupby('h_geocode').mean().reset_index()[['h_geocode', 'transit_score']].to_csv('block_transit_score_home.csv', index=False)
hw_blocks_geom_drp.groupby('w_geocode').mean().reset_index()[['w_geocode', 'transit_score']].to_csv('block_transit_score_work.csv', index=False)

In [ ]:
#aggregating transit scores for the blocks, to tract level
hw_blocks_geom_drp.groupby('home_geocode')['ratio'].mean().reset_index().to_csv('blocks_transit_ratios.csv', index=False)

## count bus stops in tracts

In [59]:
import gtfs_functions as gtfs
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./2021-10-24.zip', busiest_date=False)

In [79]:
gpd.sjoin(stops[['stop_id', 'geometry']], tracts).groupby('TRACTCE').count().reset_index()[['TRACTCE', 'stop_id']].rename({'stop_id':'stops'}, axis=1)
# .to_csv('tract_busstops.csv')

,TRACTCE,stops
0,000400,66
1,000600,17
2,000700,26
3,000800,14
4,001100,33
5,001200,48
6,001300,16
7,001400,23
8,001600,15
9,001800,32


In [90]:
hw_tracts_geom2 = hw_tracts_geom
hw_tracts_geom2['under_30_min_transit'] = hw_tracts_geom2.transit_time.apply(lambda x: 1 if x <= 1800 else 0)
hw_tracts_geom2['under_45_min_transit'] = hw_tracts_geom2.transit_time.apply(lambda x: 1 if x <= 2400 else 0)
hw_tracts_geom2['under_60_min_transit'] = hw_tracts_geom2.transit_time.apply(lambda x: 1 if x <= 3600 else 0)
# .groupby('TRACTCE20_home').mean().reset_index()[['TRACTCE20_home', 'transit_score']]

In [101]:
hw_tracts_geom2.groupby('TRACTCE20_home').sum().reset_index()[['TRACTCE20_home', 'under_30_min_transit', 'under_45_min_transit', 'under_60_min_transit']].to_csv('tracts_ts_time_levels.csv', index=False)